#### Importación de paquetes

In [21]:
import cv2
import FaceDetectors

#### Filtro que intercambia ojos y boca aplicando el detector facial basado en Convolutional Neural Networks (CNNs)

In [22]:
# Inicializa el detector facial
FDet = FaceDetectors.FaceDetector()

# Conectar a la cámara web
cap = cv2.VideoCapture(0)

# Verificar si hay otras cámaras disponibles
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Error de cam')
            exit(0)
        else:
            print('Cam 0')
    else:
        print('Cam 1')
else:
    print('Cam 0')

# Establecer la resolución de la cámara
cap.set(3, 640)
cap.set(4, 480)

# Variable que aumenta la región de los ojos y la boca
ajuste = 10

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    # Detección de rostro y ojos en el fotograma
    values = FDet.DetectLargestFaceEyesDNN(frame)
    if values is not None:
        face, eyes, shape = values

        # Dibujar un marco alrededor del rostro
        [x, y, w, h] = face
        if x > -1:
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Obtener la región de los ojos y la boca con un margen
                region_ojo_1 = frame[shape[37][1]-ajuste:shape[40][1]+ajuste, shape[36][0]-ajuste:shape[39][0]+ajuste].copy()
                region_ojo_2 = frame[shape[43][1]-ajuste:shape[46][1]+ajuste, shape[42][0]-ajuste:shape[45][0]+ajuste].copy()
                region_boca = frame[shape[50][1]-ajuste:shape[58][1]+ajuste, shape[48][0]-ajuste:shape[54][0]+ajuste].copy()

                # Verificar si las regiones tienen dimensiones válidas
                if region_ojo_1.shape[0] > 0 and region_ojo_1.shape[1] > 0 and  \
                region_ojo_2.shape[0] > 0 and region_ojo_2.shape[1] > 0 and \
                region_boca.shape[0] > 0 and region_boca.shape[1] > 0:
                    
                    # Asegurarse de que las regiones tengan las mismas dimensiones
                    region_ojos_redimensionada = cv2.resize(region_ojo_1, (region_boca.shape[1], region_boca.shape[0]))
                    region_boca_1_redimensionada = cv2.resize(region_boca, (region_ojo_1.shape[1], region_ojo_1.shape[0]))
                    region_boca_2_redimensionada = cv2.resize(region_boca, (region_ojo_2.shape[1], region_ojo_2.shape[0]))

                    # Superponer la región del ojo en la región de la boca y viceversa
                    alpha = 1  
                    superposicion1 = cv2.addWeighted(region_boca, 1 - alpha, region_ojos_redimensionada, alpha, 0)
                    superposicion2 = cv2.addWeighted(region_ojo_1, 1 - alpha, region_boca_1_redimensionada, alpha, 0)
                    superposicion3 = cv2.addWeighted(region_ojo_2, 1 - alpha, region_boca_2_redimensionada, alpha, 0)


                    frame[shape[50][1]-ajuste:shape[58][1]+ajuste, shape[48][0]-ajuste:shape[54][0]+ajuste] = superposicion1
                    frame[shape[37][1]-ajuste:shape[40][1]+ajuste, shape[36][0]-ajuste:shape[39][0]+ajuste] = superposicion2
                    frame[shape[43][1]-ajuste:shape[46][1]+ajuste, shape[42][0]-ajuste:shape[45][0]+ajuste] = superposicion3
    cv2.imshow('Cam', frame)
    
    # Esc para finalizar
    tec = cv2.waitKey(40)
    if tec == 27:  # Esc
        break

# Cerrar la ventana y liberar la cámara
cap.release()
cv2.destroyAllWindows()


Cámara 0
